In [3]:
# required libs

In [4]:
#!pip install selenium
import requests
import datetime
import json
import pandas as pd
import datetime

In [2]:
#need to download a latest chrome diver for opening whatsapp web and scanning you qr code in it from the number you want to send messages

In [25]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import itertools
import pandas as pd

# Replace below path with the absolute path
# to chromedriver in your computer

driver = webdriver.Chrome(options=chrome_options, executable_path="path--to--chrome--driver")
wait = WebDriverWait(driver, 60)
print("Chrome opened successfully!")

Web_whatsapp = 'https://web.whatsapp.com/'  # To go to whatsapp web
driver.get(Web_whatsapp)
print("Accessing Whatsapp web")


Chrome opened successfully!
Accessing Whatsapp web


In [ ]:
# to know the state and district ID run the below said code

for state_code in range(1,40):
    print("State code: ", state_code)
    response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_code))
    json_data = json.loads(response.text)
    for i in json_data["districts"]:
        print(i["district_id"],'\t', i["district_name"])
    print("\n")

In [26]:
DIST_ID = 392 # Thane
# DIST_ID = 395 # Mumbai
# DIST_ID = 706 #Pithoragarh

# Print available centre description (y/n)?
print_flag = 'y'

numdays = 7 # will check the status of next 7 days
age = 55 # defaul age of person

In [27]:

base = datetime.datetime.today()
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
date_str = [x.strftime("%d-%m-%Y") for x in date_list]

In [28]:


def get_slot(age,date_str,DIST_ID):
    slot_lst=[]
    slot_lst.append(['For Age:',age,'starting date for next 20 days:',date_str[0], 'for location:',DIST_ID,'(Thane-392,Mumbai-395)'])
    for INP_DATE in date_str:
        URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, INP_DATE)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
        response = requests.get(URL,headers=headers)
        if response.ok:
            resp_json = response.json()
            #print(resp_json['centers'])
            if resp_json['centers']:
                slot_lst.append("Available Status on: {}".format(INP_DATE))
                if(print_flag=='y' or print_flag=='Y'):
                    for center in resp_json['centers']:
                        for session in center["sessions"]:
                            if session["min_age_limit"] <= age and session["available_capacity"]>0:
                                slot_lst.append([center["name"].upper(),center["block_name"],"Price: "+center["fee_type"],"AVAILABLE CAPACITY>>>"+str(session["available_capacity"])])
                                if(session["vaccine"] != ''):
                                    slot_lst.append("Vaccine: "+session["vaccine"])
            else:
                slot_lst.append("No available slots on {}".format(INP_DATE))
        else:
            return 'No Response from COWIN Site'

    return '\n'.join(str(v) for v in slot_lst)

In [29]:
i = 'Vaccination_Alerts' # here put name of the group or contact on your whatsapp contact list...it should be in your chat list...it will not create new chat
j = 'All'
k = 'Hi message from python'

def sendwatsappmsg(i,j,k):
    Search = wait.until(EC.element_to_be_clickable((By.XPATH,"//div[@class='_2_1wd copyable-text selectable-text']")))
    time.sleep(2)

    Search.send_keys(i)
    time.sleep(1)
    Search.send_keys(Keys.ENTER)
    time.sleep(3)

    Message = driver.find_elements_by_xpath('//*[@id="main"]/footer/div[1]/div[2]/div/div[2]')[0]
    time.sleep(2)

    Message.send_keys('Hello ' + str(j) + ',')
    time.sleep(1)
    Message.send_keys(Keys.SHIFT, Keys.ENTER)
    Message.send_keys(Keys.SHIFT, Keys.ENTER)
    Message.send_keys('This is the LIVE status of *COWIN* Vaccination \n' + str(k))
    time.sleep(1)
    Message.send_keys(Keys.ENTER)
    print('Sent to ' + str(j))


In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

def some_job():
    age=30 # this is for age you or your family member is
    DIST_ID = 392 #district code as obtained above
    young_thane = get_slot(age,date_str,DIST_ID)
    sendwatsappmsg(i,j,young_thane)
    time.sleep(10)
    age=50
    DIST_ID = 392
    old_thane = get_slot(age,date_str,DIST_ID)
    sendwatsappmsg(i,j,old_thane)
    time.sleep(10)
    age=30
    DIST_ID = 395
    young_mum = get_slot(age,date_str,DIST_ID)
    sendwatsappmsg(i,j,young_mum)
    time.sleep(10)
    age=50
    DIST_ID = 395
    old_mum = get_slot(age,date_str,DIST_ID)
    sendwatsappmsg(i,j,old_mum)
    time.sleep(10)

scheduler = BlockingScheduler()
scheduler.add_job(some_job, 'interval', minutes=45) #here the code will check the status every 45 mins and send an update on the whatsapp
scheduler.start()